In [1]:
import pandas as pd
import numpy as np
import os

# --- 0. Configuración ---
# Crea la carpeta 'predictions' si no existe
output_folder = "predictions"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# --- 1. Cargar el Dataset ---
# ¡IMPORTANTE!: Asegúrate de que 'datasetwg.csv' esté en la misma carpeta que este script,
# o proporciona la ruta completa al archivo (ej. "C:/Users/TuUsuario/Documentos/datasetwg.csv").
try:
    df_raw = pd.read_csv("datasetwg.csv")
    print("Dataset 'datasetwg.csv' cargado exitosamente.")
    print(f"Columnas del dataset: {df_raw.columns.tolist()}")
except FileNotFoundError:
    print("¡ERROR!: No se encontró el archivo 'datasetwg.csv'. Por favor, asegúrate de que la ruta sea correcta.")
    print("El script continuará con un DataFrame de ejemplo, pero los resultados no serán sobre tus datos reales.")
    # Creando un DataFrame de ejemplo para que el script no falle inmediatamente
    data = {
        'periodo': pd.to_datetime(['2017-01-01', '2017-02-01', '2019-10-01', '2019-11-01', '2019-12-01'] * 5),
        'customer_id': ['C101', 'C102', 'C103', 'C101', 'C102'] * 5,
        'product_id': ['P001', 'P002', 'P001', 'P003', 'P002'] * 5,
        'tn': np.random.randint(10, 1000, 25)
    }
    df_raw = pd.DataFrame(data)
    # Ajustar periodos para el ejemplo
    df_raw['periodo'] = df_raw['periodo'].dt.to_period('M') # Convertir a PeriodDtype para facilitar comparaciones de meses


# --- 2. Preprocesamiento de Datos ---
# Convertir la columna 'periodo' a formato de fecha si no lo está
# Asumimos que 'periodo' está en un formato que pandas puede interpretar (YYYY-MM-DD o YYYY-MM)
df_raw['periodo'] = pd.to_datetime(df_raw['periodo'])
df_raw['periodo'] = df_raw['periodo'].dt.to_period('M') # Convertir a PeriodDtype para facilitar comparaciones de meses

# Agrupar las ventas (tn) por periodo y product_id, sumando todos los customer_id
df_agg = df_raw.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

# Ordenar por periodo para asegurar cálculos de series temporales correctos
df_agg = df_agg.sort_values(by=['periodo', 'product_id']).reset_index(drop=True)

# Obtener el último periodo disponible (Diciembre 2019)
last_period_data = df_agg['periodo'].max()
print(f"Último periodo de datos disponible: {last_period_data}")

# Definir el periodo de predicción (Febrero 2020)
prediction_period = pd.Period('2020-02', freq='M')

# --- 3. Implementación de Predicciones Triviales ---

# Función para guardar los resultados
def save_prediction(df_prediction, filename):
    filepath = os.path.join(output_folder, filename)
    df_prediction.to_csv(filepath, index=False)
    print(f"Predicción guardada en: {filepath}")

# 3.1. Predicción: Último mes (Diciembre 2019)
print("\nCalculando Predicción: Último mes (Dic 2019)...")
df_last_month = df_agg[df_agg['periodo'] == last_period_data].copy()
df_last_month = df_last_month[['product_id', 'tn']].rename(columns={'tn': 'predicted_tn_feb_2020'})
save_prediction(df_last_month, "prediction_last_month_sales_feb_2020.csv")

# 3.2. Predicción: Promedio de los últimos 3 meses (Oct-Dic 2019)
print("\nCalculando Predicción: Promedio de los últimos 3 meses (Oct-Dic 2019)...")
# Calcular la fecha límite inferior para los 3 meses
end_date_3m = pd.Period('2019-12', freq='M')
start_date_3m = pd.Period('2019-10', freq='M')

df_3m_avg = df_agg[(df_agg['periodo'] >= start_date_3m) & (df_agg['periodo'] <= end_date_3m)]
# Agrupar por product_id y calcular el promedio de 'tn'
prediction_3m_avg = df_3m_avg.groupby('product_id')['tn'].mean().reset_index()
prediction_3m_avg.rename(columns={'tn': 'predicted_tn_feb_2020'}, inplace=True)
save_prediction(prediction_3m_avg, "prediction_avg_last_3m_sales_feb_2020.csv")

# 3.3. Predicción: Promedio de los últimos 6 meses (Jul-Dic 2019)
print("\nCalculando Predicción: Promedio de los últimos 6 meses (Jul-Dic 2019)...")
end_date_6m = pd.Period('2019-12', freq='M')
start_date_6m = pd.Period('2019-07', freq='M')

df_6m_avg = df_agg[(df_agg['periodo'] >= start_date_6m) & (df_agg['periodo'] <= end_date_6m)]
prediction_6m_avg = df_6m_avg.groupby('product_id')['tn'].mean().reset_index()
prediction_6m_avg.rename(columns={'tn': 'predicted_tn_feb_2020'}, inplace=True)
save_prediction(prediction_6m_avg, "prediction_avg_last_6m_sales_feb_2020.csv")

# 3.4. Predicción: Promedio de los últimos 9 meses (Abr-Dic 2019)
print("\nCalculando Predicción: Promedio de los últimos 9 meses (Abr-Dic 2019)...")
end_date_9m = pd.Period('2019-12', freq='M')
start_date_9m = pd.Period('2019-04', freq='M')

df_9m_avg = df_agg[(df_agg['periodo'] >= start_date_9m) & (df_agg['periodo'] <= end_date_9m)]
prediction_9m_avg = df_9m_avg.groupby('product_id')['tn'].mean().reset_index()
prediction_9m_avg.rename(columns={'tn': 'predicted_tn_feb_2020'}, inplace=True)
save_prediction(prediction_9m_avg, "prediction_avg_last_9m_sales_feb_2020.csv")

# 3.5. Predicción: Promedio de los últimos 12 meses (Ene-Dic 2019)
print("\nCalculando Predicción: Promedio de los últimos 12 meses (Ene-Dic 2019)...")
end_date_12m = pd.Period('2019-12', freq='M')
start_date_12m = pd.Period('2019-01', freq='M')

df_12m_avg = df_agg[(df_agg['periodo'] >= start_date_12m) & (df_agg['periodo'] <= end_date_12m)]
prediction_12m_avg = df_12m_avg.groupby('product_id')['tn'].mean().reset_index()
prediction_12m_avg.rename(columns={'tn': 'predicted_tn_feb_2020'}, inplace=True)
save_prediction(prediction_12m_avg, "prediction_avg_last_12m_sales_feb_2020.csv")

# --- Funciones para Media Móvil Exponencial (EMA) ---
# La EMA se calcula generalmente sobre una serie temporal. Para cada product_id,
# necesitamos calcular la EMA de su historial de ventas 'tn'.

def calculate_ema_prediction(df_data, periods, span_value, name_suffix):
    print(f"\nCalculando Predicción: EMA de los últimos {periods} meses (span={span_value})...")
    # Para cada product_id, aplicar la EMA
    # Primero, necesitamos asegurarnos de que tenemos todos los meses para cada product_id si es necesario,
    # o que la ventana de EMA solo considere los meses existentes.
    # Aquí, calculamos la EMA por product_id y luego tomamos el último valor disponible de la EMA.

    # Obtener el historial para el cálculo de la EMA
    if periods == 3:
        ema_df = df_data[(df_data['periodo'] >= pd.Period('2019-10', freq='M')) & (df_data['periodo'] <= pd.Period('2019-12', freq='M'))]
    elif periods == 6:
        ema_df = df_data[(df_data['periodo'] >= pd.Period('2019-07', freq='M')) & (df_data['periodo'] <= pd.Period('2019-12', freq='M'))]
    elif periods == 9:
        ema_df = df_data[(df_data['periodo'] >= pd.Period('2019-04', freq='M')) & (df_data['periodo'] <= pd.Period('2019-12', freq='M'))]
    elif periods == 12:
        ema_df = df_data[(df_data['periodo'] >= pd.Period('2019-01', freq='M')) & (df_data['periodo'] <= pd.Period('2019-12', freq='M'))]
    else:
        print(f"Error: Número de periodos ({periods}) no soportado para EMA. Debe ser 3, 6, 9 o 12.")
        return pd.DataFrame()

    # Si hay product_id sin datos en el rango, no se calculará EMA para ellos.
    # Para EMA, necesitamos el último valor calculado.
    # Agrupamos por product_id, ordenamos por periodo y aplicamos ewm.
    prediction_ema = ema_df.groupby('product_id')['tn'].apply(lambda x: x.ewm(span=span_value, adjust=False).mean().iloc[-1] if not x.empty else np.nan).reset_index()
    prediction_ema.rename(columns={'tn': 'predicted_tn_feb_2020'}, inplace=True)
    # Eliminar NaN si algún product_id no tiene suficientes datos para el cálculo
    prediction_ema.dropna(subset=['predicted_tn_feb_2020'], inplace=True)
    save_prediction(prediction_ema, f"prediction_ema_last_{periods}m_sales_feb_2020{name_suffix}.csv")

# 3.6. Predicción: Media Móvil Exponencial (EMA) de los últimos 3 meses
# 'span' se refiere al número de períodos de la ventana de la EMA.
calculate_ema_prediction(df_agg, 3, 3, "_span3")

# 3.7. Predicción: Media Móvil Exponencial (EMA) de los últimos 6 meses
calculate_ema_prediction(df_agg, 6, 6, "_span6")

# 3.8. Predicción: Media Móvil Exponencial (EMA) de los últimos 9 meses
calculate_ema_prediction(df_agg, 9, 9, "_span9")

# 3.9. Predicción: Media Móvil Exponencial (EMA) de los últimos 12 meses
calculate_ema_prediction(df_agg, 12, 12, "_span12")

print("\nTodas las predicciones triviales han sido generadas y guardadas en la carpeta 'predictions'.")

Dataset 'datasetwg.csv' cargado exitosamente.
Columnas del dataset: ['customer_id', 'product_id', 'periodo', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'venta_id', 'DIFF(cust_request_qty)', 'DIFF(cust_request_tn)', 'DIFF(plan_precios_cuidados)', 'DIFF(sku_size)', 'DIFF(stock_final)', 'DIFF(tn)', 'LAG(brand, periodo)', 'LAG(cat1, periodo)', 'LAG(cat2, periodo)', 'LAG(cat3, periodo)', 'LAG(cust_request_qty, periodo)', 'LAG(cust_request_tn, periodo)', 'LAG(plan_precios_cuidados, periodo)', 'LAG(sku_size, periodo)', 'LAG(stock_final, periodo)', 'LAG(tn, periodo)', 'MONTH(periodo)', 'ROLLING_MEAN(periodo, cust_request_qty)', 'ROLLING_MEAN(periodo, cust_request_tn)', 'ROLLING_MEAN(periodo, plan_precios_cuidados)', 'ROLLING_MEAN(periodo, sku_size)', 'ROLLING_MEAN(periodo, stock_final)', 'ROLLING_MEAN(periodo, tn)', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3', 'tn_lag_4', 'tn_lag_5', 'tn_delta']
Último periodo de dat

In [2]:
import pandas as pd
import os

# --- 0. Configuración ---
# La carpeta donde se guardaron las predicciones individuales del script anterior.
# Asegúrate de que esta carpeta exista y contenga los CSVs de las predicciones.
output_folder = "predictions" 

# --- 1. Ensemble entre los Promedios ---
print("Realizando ensemble entre los promedios...")

# Lista de archivos de predicción basados en promedios
avg_files = [
    "prediction_avg_last_3m_sales_feb_2020.csv",
    "prediction_avg_last_6m_sales_feb_2020.csv",
    "prediction_avg_last_9m_sales_feb_2020.csv",
    "prediction_avg_last_12m_sales_feb_2020.csv"
]

# DataFrame para almacenar todas las predicciones de promedio
all_avg_predictions = pd.DataFrame()

for file_name in avg_files:
    file_path = os.path.join(output_folder, file_name)
    if os.path.exists(file_path):
        df_temp = pd.read_csv(file_path)
        # Renombrar la columna de predicción para evitar conflictos al hacer merge
        # Usamos un nombre único basado en el nombre del archivo original
        df_temp.rename(columns={'predicted_tn_feb_2020': f'pred_{file_name.replace(".csv", "")}'}, inplace=True)
        
        if all_avg_predictions.empty:
            # Si es el primer archivo, inicializar el DataFrame
            all_avg_predictions = df_temp[['product_id', f'pred_{file_name.replace(".csv", "")}']]
        else:
            # Para los archivos subsiguientes, hacer un merge por 'product_id'
            all_avg_predictions = pd.merge(all_avg_predictions, df_temp[['product_id', f'pred_{file_name.replace(".csv", "")}']], on='product_id', how='outer')
    else:
        print(f"Advertencia: No se encontró el archivo {file_name} en '{output_folder}'. Se omitirá en el ensemble de promedios.")

if not all_avg_predictions.empty:
    # Obtener solo las columnas que contienen las predicciones (las que empiezan con 'pred_')
    prediction_columns = [col for col in all_avg_predictions.columns if col.startswith('pred_')]
    # Calcular el promedio de esas columnas para cada product_id
    all_avg_predictions['ensemble_avg_tn_feb_2020'] = all_avg_predictions[prediction_columns].mean(axis=1)

    # Seleccionar solo 'product_id' y la nueva columna de ensemble final
    ensemble_avg_result = all_avg_predictions[['product_id', 'ensemble_avg_tn_feb_2020']]

    # Guardar el resultado del ensemble de promedios
    ensemble_avg_file_path = os.path.join(output_folder, "ensemble_average_predictions_feb_2020.csv")
    ensemble_avg_result.to_csv(ensemble_avg_file_path, index=False)
    print(f"Ensemble de promedios guardado en: {ensemble_avg_file_path}")
else:
    print("No se pudieron cargar suficientes archivos de promedio para realizar el ensemble. Verifique la existencia de los archivos.")

# --- 2. Ensemble entre las EMAs ---
print("\nRealizando ensemble entre las EMAs...")

# Lista de archivos de predicción basados en EMAs
ema_files = [
    "prediction_ema_last_3m_sales_feb_2020_span3.csv",
    "prediction_ema_last_6m_sales_feb_2020_span6.csv",
    "prediction_ema_last_9m_sales_feb_2020_span9.csv",
    "prediction_ema_last_12m_sales_feb_2020_span12.csv"
]

# DataFrame para almacenar todas las predicciones de EMA
all_ema_predictions = pd.DataFrame()

for file_name in ema_files:
    file_path = os.path.join(output_folder, file_name)
    if os.path.exists(file_path):
        df_temp = pd.read_csv(file_path)
        # Renombrar la columna de predicción para evitar conflictos al hacer merge
        df_temp.rename(columns={'predicted_tn_feb_2020': f'pred_{file_name.replace(".csv", "")}'}, inplace=True)
        
        if all_ema_predictions.empty:
            all_ema_predictions = df_temp[['product_id', f'pred_{file_name.replace(".csv", "")}']]
        else:
            all_ema_predictions = pd.merge(all_ema_predictions, df_temp[['product_id', f'pred_{file_name.replace(".csv", "")}']], on='product_id', how='outer')
    else:
        print(f"Advertencia: No se encontró el archivo {file_name} en '{output_folder}'. Se omitirá en el ensemble de EMAs.")

if not all_ema_predictions.empty:
    # Obtener solo las columnas que contienen las predicciones
    prediction_columns = [col for col in all_ema_predictions.columns if col.startswith('pred_')]
    # Calcular el promedio de esas columnas para cada product_id
    all_ema_predictions['ensemble_ema_tn_feb_2020'] = all_ema_predictions[prediction_columns].mean(axis=1)

    # Seleccionar solo 'product_id' y la nueva columna de ensemble final
    ensemble_ema_result = all_ema_predictions[['product_id', 'ensemble_ema_tn_feb_2020']]

    # Guardar el resultado del ensemble de EMAs
    ensemble_ema_file_path = os.path.join(output_folder, "ensemble_ema_predictions_feb_2020.csv")
    ensemble_ema_result.to_csv(ensemble_ema_file_path, index=False)
    print(f"Ensemble de EMAs guardado en: {ensemble_ema_file_path}")
else:
    print("No se pudieron cargar suficientes archivos de EMA para realizar el ensemble. Verifique la existencia de los archivos.")

print("\nProceso de ensemble finalizado.")

Realizando ensemble entre los promedios...
Ensemble de promedios guardado en: predictions\ensemble_average_predictions_feb_2020.csv

Realizando ensemble entre las EMAs...
Ensemble de EMAs guardado en: predictions\ensemble_ema_predictions_feb_2020.csv

Proceso de ensemble finalizado.


In [3]:
import pandas as pd
import numpy as np
import os

# --- 0. Configuración ---
# Crea la carpeta 'predictions' si no existe
output_folder = "predictions"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# --- 1. Cargar el Dataset ---
# Asegúrate de que 'datasetwg.csv' esté en la misma carpeta que este script,
# o proporciona la ruta completa al archivo (ej. "C:/Users/TuUsuario/Documentos/datasetwg.csv").
try:
    df_raw = pd.read_csv("datasetwg.csv")
    print("Dataset 'datasetwg.csv' cargado exitosamente.")
except FileNotFoundError:
    print("¡ERROR!: No se encontró el archivo 'datasetwg.csv'. Por favor, asegúrate de que la ruta sea correcta.")
    print("El script continuará con un DataFrame de ejemplo para fines demostrativos.")
    # Creando un DataFrame de ejemplo si el archivo no se encuentra
    data = {
        'periodo': pd.to_datetime([f'2018-{m:02d}-01' for m in range(7, 13)] + [f'2019-{m:02d}-01' for m in range(1, 13)] * 2), # 18 meses
        'customer_id': ['C101', 'C102', 'C103'] * 12,
        'product_id': ['P001', 'P002', 'P003', 'P001', 'P002', 'P003'] * 6,
        'tn': np.random.randint(50, 1500, 36)
    }
    df_raw = pd.DataFrame(data)
    df_raw['periodo'] = df_raw['periodo'].dt.to_period('M')


# --- 2. Preprocesamiento de Datos ---
# Convertir la columna 'periodo' a formato de fecha si no lo está
df_raw['periodo'] = pd.to_datetime(df_raw['periodo'])
df_raw['periodo'] = df_raw['periodo'].dt.to_period('M') # Convertir a PeriodDtype para facilitar comparaciones

# Agrupar las ventas (tn) por periodo y product_id, sumando todos los customer_id
df_agg = df_raw.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

# Ordenar por periodo para asegurar cálculos de series temporales correctos
df_agg = df_agg.sort_values(by=['periodo', 'product_id']).reset_index(drop=True)


# --- 3. Predicción: Promedio de los últimos 18 meses (Jul 2018 - Dic 2019) ---
print("\nCalculando Predicción: Promedio de los últimos 18 meses (Jul 2018 - Dic 2019)...")

# Definir el rango de fechas para los últimos 18 meses
end_date_18m = pd.Period('2019-12', freq='M')
start_date_18m = pd.Period('2018-07', freq='M') # 18 meses antes de 2019-12

df_18m_avg = df_agg[(df_agg['periodo'] >= start_date_18m) & (df_agg['periodo'] <= end_date_18m)]

if not df_18m_avg.empty:
    # Agrupar por product_id y calcular el promedio de 'tn'
    prediction_18m_avg = df_18m_avg.groupby('product_id')['tn'].mean().reset_index()
    prediction_18m_avg.rename(columns={'tn': 'predicted_tn_feb_2020'}, inplace=True)

    # Guardar el resultado
    filepath_18m_avg = os.path.join(output_folder, "prediction_avg_last_18m_sales_feb_2020.csv")
    prediction_18m_avg.to_csv(filepath_18m_avg, index=False)
    print(f"Predicción de promedio de los últimos 18 meses guardada en: {filepath_18m_avg}")
else:
    print(f"No se encontraron datos para el rango de {start_date_18m} a {end_date_18m}. No se pudo calcular la predicción.")

print("\nProceso de predicción finalizado.")

Dataset 'datasetwg.csv' cargado exitosamente.

Calculando Predicción: Promedio de los últimos 18 meses (Jul 2018 - Dic 2019)...
Predicción de promedio de los últimos 18 meses guardada en: predictions\prediction_avg_last_18m_sales_feb_2020.csv

Proceso de predicción finalizado.
